In [22]:
import os
import cv2
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix

from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

In [23]:
#  Load MNIST data 
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [24]:
# Declare model/data parameters
input_shape = (28, 28, 1)
batch_size = 64
num_classes = 10
epochs = 10

In [25]:
# Preprocess the data
# Normalize all image data
x_train = x_train / 255.0
x_test = x_test / 255.0

# Reshape the data for the input shape (28,28,1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# One-hot encoding for the labels
y_train = tf.one_hot(y_train.astype(np.int32), depth=10)
y_test = tf.one_hot(y_test.astype(np.int32), depth=10)

In [26]:
# Build the model
model = tf.keras.Sequential(
    [
        keras.Input(shape=input_shape),
        tf.keras.layers.Conv2D(32, (5,5), padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ]
)

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 28, 28, 32)        832       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 14, 14, 32)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 14, 14, 32)        9248      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 7, 7, 32)         0         
 2D)                                                             
                                                                 
 dropout_7 (Dropout)         (None, 7, 7, 32)         

In [27]:
# Compile and train the model
model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=["accuracy"])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=0.1)

Epoch 1/10
844/844 [==============================] - 5s 6ms/step - loss: 0.3113 - accuracy: 0.9016 - val_loss: 0.0644 - val_accuracy: 0.9817
Epoch 2/10
844/844 [==============================] - 4s 4ms/step - loss: 0.1158 - accuracy: 0.9648 - val_loss: 0.0445 - val_accuracy: 0.9885
Epoch 3/10
844/844 [==============================] - 4s 4ms/step - loss: 0.0880 - accuracy: 0.9732 - val_loss: 0.0390 - val_accuracy: 0.9882
Epoch 4/10
844/844 [==============================] - 4s 5ms/step - loss: 0.0772 - accuracy: 0.9764 - val_loss: 0.0335 - val_accuracy: 0.9912
Epoch 5/10
844/844 [==============================] - 4s 4ms/step - loss: 0.0665 - accuracy: 0.9799 - val_loss: 0.0316 - val_accuracy: 0.9913
Epoch 6/10
844/844 [==============================] - 4s 4ms/step - loss: 0.0605 - accuracy: 0.9817 - val_loss: 0.0327 - val_accuracy: 0.9902
Epoch 7/10
844/844 [==============================] - 4s 4ms/step - loss: 0.0530 - accuracy: 0.9832 - val_loss: 0.0333 - val_accuracy: 0.9905
Epoch 

In [28]:
# Evaluate the trained model on MNIST test data
test_loss, test_acc = model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.0215 - accuracy: 0.9922


In [29]:
# Prepare the custom dataset made by me
drc = r"/content/drive/MyDrive/datasets/numbers_digital_labeled_28px"
CATEGORIES = ["0", "1","2", "3","4", "5","6", "7","8", "9"]

numbers = []
for category in CATEGORIES:  
    path = os.path.join(drc,category) 
    class_num = CATEGORIES.index(category)
    for img in os.listdir(path):  
        img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
        numbers.append([img_array, class_num])

In [30]:
# Process the custom dataset, first shuffle and then split to images and labes array. 
# Reshape and normalize images, and one-hot encode the labels
random.shuffle(numbers)

x_numbers = []
y_numbers = []

for features, label in numbers:
    x_numbers.append(features)
    y_numbers.append(label)

y_numbers = np.asarray(y_numbers)
    
x_numbers = np.array(x_numbers).reshape(-1, 28, 28, 1)
x_numbers = x_numbers/255

y_numbers = tf.one_hot(y_numbers.astype(np.int32), depth=10)

In [31]:
# Evaluate the trained model on custom data
numbers_loss, numbers_acc = model.evaluate(x_numbers, y_numbers)

2/2 [==============================] - 0s 40ms/step - loss: 1.9489 - accuracy: 0.5893


In [32]:
# Get predictions for test data and custom data.
# Decode one-hot groundtruth labels and predicted labels for test data and custom data.
test_predictions = model.predict(x_test)
number_predictions = model.predict(x_numbers)

groundtruth_test_labels = tf.argmax(y_test, axis=1)
groundtruth_number_labels = tf.argmax(y_numbers, axis=1)

predicted_test_labels = tf.argmax(test_predictions, axis=1)
predicted_number_labels = tf.argmax(number_predictions, axis=1)

In [33]:
 # Print confusion matrices.
confusion_matrix_test = confusion_matrix(groundtruth_test_labels,predicted_test_labels)
print("confusion_matrix_test")
print(confusion_matrix_test)

confusion_matrix_numbers = confusion_matrix(groundtruth_number_labels,predicted_number_labels)
print("confusion_matrix_numbers")
print(confusion_matrix_numbers)

confusion_matrix_test
[[ 978    0    0    0    0    0    0    1    1    0]
 [   0 1128    1    0    0    1    2    1    2    0]
 [   1    0 1028    0    0    0    0    3    0    0]
 [   0    0    1 1001    0    6    0    1    1    0]
 [   0    0    0    0  979    0    0    0    2    1]
 [   1    0    0    2    0  886    1    0    0    2]
 [   7    2    1    0    2    6  940    0    0    0]
 [   0    2    4    1    0    0    0 1018    1    2]
 [   1    0    1    1    0    1    0    1  968    1]
 [   1    0    0    0    3    3    0    3    3  996]]
confusion_matrix_numbers
[[1 0 0 0 0 0 0 0 0 4]
 [0 5 0 0 0 0 0 0 0 0]
 [0 0 5 0 0 0 0 0 0 0]
 [0 0 0 4 0 0 0 1 0 1]
 [0 1 0 0 6 0 0 0 0 0]
 [0 0 0 0 1 3 0 0 0 1]
 [1 0 0 0 2 0 0 0 0 3]
 [0 0 0 1 0 1 0 4 0 0]
 [0 0 1 0 1 0 1 0 2 0]
 [0 0 0 0 0 0 0 1 2 3]]
